In [ ]:
#
# Script meant to test the ability to show brief "help" animations in the interactive graph panel
# ... animations still showing incorrect behavior
#
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
g         = rt.createNetworkXGraph(df, _relates_)

# Provides colors based on community
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)

#_rtg_ = rt.interactiveGraphPanel(df, {'relationships':_relates_, 'node_color':_node_colors_}, w=800, h=500)
#_rtg_

In [ ]:
#_rtg_.setAnimation('<rect x="10" y="10" width="10" height="10" fill="blue"/>')

In [ ]:
#_rtg_.setAnimation('<text x="5" y="15" fill="black"> popStack </text>')

In [ ]:
from rtsvg.polars_force_directed_layout import PolarsForceDirectedLayout
_pfdl_ = PolarsForceDirectedLayout(g, k=1)
rt.link(df, _relates_, pos=_pfdl_.results(), node_color=_node_colors_)

In [ ]:
# The first element is garbage ... so throw away a few of them...
k0, k1, k2 = _pfdl_.stressVector(k=0)[16:], _pfdl_.stressVector(k=1)[16:], _pfdl_.stressVector(k=2)[16:]
df0, df1, df2 = pl.DataFrame({'x':[i for i in range(len(k0))], 'y':k0}), pl.DataFrame({'x':[i for i in range(len(k1))], 'y':k1}), pl.DataFrame({'x':[i for i in range(len(k2))], 'y':k2})
_params_ = {'x_field':'x', 'y_field':'y'}
rt.tile([rt.xy(df0, **_params_), rt.xy(df1, **_params_), rt.xy(df2, **_params_)])